In [21]:
import requests
import json
import time
import pandas as pd

In [17]:
league_id = 71 # BR League
season = 2022
now = time.time()

In [3]:
headers = {
    'X-RapidAPI-Key': 'a7f6ca7698mshc1bf4dfa3fd55dfp107570jsn41aa3143cd7b',
    'X-RapidAPI-Host': 'api-football-v1.p.rapidapi.com'
}

In [4]:
response = requests.get(f"https://api-football-v1.p.rapidapi.com/v3/fixtures?league={league_id}&season={season}", headers=headers)

In [7]:
response_parsed = json.loads(response.text)

In [19]:
fixtures = [fixture for fixture in response_parsed['response'] if fixture['fixture']['timestamp'] < now]
fixtures_data = []
fixtures_cols = ['GAME_DATE', 'HOME_ID', 'HOME_NAME', 'AWAY_ID', 'AWAY_NAME', 'HOME_SCORE', 'AWAY_SCORE']
for fixture in fixtures:
    fixtures_data.append([fixture['fixture']['timestamp'], fixture['teams']['home']['id'], fixture['teams']['home']['name'], fixture['teams']['away']['id'], fixture['teams']['away']['name'], fixture['goals']['home'], fixture['goals']['away']])
    print(f"{fixture['teams']['home']['name']} {fixture['goals']['home']} x {fixture['goals']['away']} {fixture['teams']['away']['name']}")

Atletico-MG 2 x 0 Internacional
Fluminense 0 x 0 Santos
Sao Paulo 4 x 0 Atletico Paranaense
Palmeiras 2 x 3 Ceara
Botafogo 1 x 3 Corinthians
Juventude 2 x 2 RB Bragantino
Fortaleza EC 0 x 1 Cuiaba
Atletico Goianiense 1 x 1 Flamengo
Avai 1 x 0 America Mineiro
Coritiba 3 x 0 Goias
America Mineiro 4 x 1 Juventude
Flamengo 3 x 1 Sao Paulo
RB Bragantino 4 x 0 Atletico Goianiense
Corinthians 3 x 0 Avai
Santos 2 x 1 Coritiba
Internacional 2 x 1 Fortaleza EC
Ceara 1 x 3 Botafogo
Goias 1 x 1 Palmeiras
Cuiaba 0 x 1 Fluminense
Atletico Paranaense 0 x 1 Atletico-MG
Atletico-MG 2 x 2 Coritiba
Fluminense 0 x 1 Internacional
RB Bragantino 1 x 1 Sao Paulo
Palmeiras 3 x 0 Corinthians
Santos 3 x 0 America Mineiro
Juventude 0 x 1 Cuiaba
Fortaleza EC 0 x 1 Ceara
Atletico Goianiense 1 x 1 Botafogo
Avai 3 x 2 Goias
Atletico Paranaense 1 x 0 Flamengo
America Mineiro 1 x 0 Atletico Paranaense
Flamengo 0 x 0 Palmeiras
Sao Paulo 2 x 1 Santos
Corinthians 1 x 0 Fortaleza EC
Botafogo 1 x 1 Juventude
Internacional 

In [23]:
fixtures_df = pd.DataFrame(fixtures_data, columns = fixtures_cols)

In [24]:
fixtures_df.head()

,GAME_DATE,HOME_ID,HOME_NAME,AWAY_ID,AWAY_NAME,HOME_SCORE,AWAY_SCORE
0,1649617200,1062,Atletico-MG,119,Internacional,2,0
1,1649532600,124,Fluminense,128,Santos,0,0
2,1649628000,126,Sao Paulo,134,Atletico Paranaense,4,0
3,1649548800,121,Palmeiras,129,Ceara,2,3
4,1649617200,120,Botafogo,131,Corinthians,1,3
